In [25]:
from sys import exit
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes
import random
import string
from pwn import *
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits

## **Bài toán EVEN RSA CAN BE BROKEN???**

Khi phân tích code của bài toán ta có thể thấy, tại phương thức `main()`, ta thấy răng N là tích của hai số nguyên tố với kích thước 512 bit.
~~~python
def gen_key(k):
    """
    Generates RSA key with k bits
    """
    p,q = get_primes(k//2)
    N = p*q
    d = inverse(e, (p-1)*(q-1))

    return ((N,e), d)
~~~
Như vậy, N là một số RSA yếu, chỉ có 1024bit, do đó, ta sử dụng các hàm có thể phân tách N.

In [ ]:
# Sử dụng thư viện primefac
import primefac

N = 17968197884504104747  # Thay bằng N của bạn
factors = list(primefac.primefac(N))
print("Thừa số nguyên tố của N:", factors)

Thừa số nguyên tố của N: [29, 6438077, 96238835059]


In [6]:
# Sử dụng truy vấn đến API bên ngoài
import requests

N = 17968197884504104747 
url = f"http://factordb.com/api?query={N}"
response = requests.get(url).json()

factors = response.get("factors", [])
print("Thừa số nguyên tố của N:", factors)

Thừa số nguyên tố của N: [['29', 1], ['6438077', 1], ['96238835059', 1]]


### **Lời giải**
#### **Đầu tiên ta ncat đến server để lấy giá trị của N**

In [ ]:
port = 52407
r = remote('verbal-sleep.picoctf.net', port)

N = int(r.recvline(keepends=False).strip().decode().strip("N: "))
e = int(r.recvline(keepends=False).strip().decode().strip("e: "))
cyphertext = int(r.recvline(keepends=False).strip().decode().strip("cyphertext: "))

print(f"N: {N}") # N chẵn

# Tìm thừa số nguyên tố của N
factors = list(primefac.primefac(N))
print("Thừa số nguyên tố của N:", factors)
p, q = factors[0], factors[1]
d = inverse(e, (p-1)*(q-1))
plaintext = pow(cyphertext, d, N)
flag = long_to_bytes(plaintext).decode('utf-8')
print(flag)

[x] Opening connection to verbal-sleep.picoctf.net on port 52407
[x] Opening connection to verbal-sleep.picoctf.net on port 52407: Trying 3.138.217.147
[+] Opening connection to verbal-sleep.picoctf.net on port 52407: Done
N: 15709310530033121584347799651253894683012237012174722800716346125443382011330008779902493365517209294329072981514221961360754564548158849509561108729716166
Thừa số nguyên tố của N: [2, 7854655265016560792173899825626947341506118506087361400358173062721691005665004389951246682758604647164536490757110980680377282274079424754780554364858083]
picoCTF{tw0_1$_pr!m3605cd50e}
